In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy.stats import norm, linregress

import os
import time
import itertools

from glove_VI.glv3 import *

from sklearn.model_selection import KFold

In [ ]:
# number of folds
n_splits = 20

# number of inner folds
n_splits_2 = 10

# range of L1 coefs
lmbdas = np.array([0., .000001, .00001, .0001, .001])

# number of training epochs
n_epochs = 200

# import file names
files = os.listdir("data/SET3_Thirdtrial/")

# fit gLV models

In [ ]:
def predict_df(model, df, species):
    
    # save measured and predicted values
    exp_names = []
    pred_species = []
    pred = []
    true = []

    # pull just the community data
    test_data = process_df_glove(df, species) 

    # plot the results
    for exp, t_span, Y_m in test_data:

        # predict 
        Y_p = model.predict_point(Y_m, t_span)
        
        # set NaN to zero
        Y_p = np.nan_to_num(Y_p)
        
        ### prediction results for species that were present ###
        inds_present = Y_m[0] > 0 
        exp_names.append([exp]*sum(inds_present)*(Y_m.shape[0]-1))
        pred_species.append(np.tile(np.vstack(species)[inds_present], Y_m.shape[0]-1).T.ravel())
        true.append(Y_m[1:,inds_present].ravel())
        pred.append(Y_p[1:,inds_present].ravel())
                
    # concatenate list
    exp_names = np.concatenate(exp_names)
    pred_species = np.concatenate(pred_species)
    true = np.concatenate(true)
    pred = np.concatenate(pred)
        
    return exp_names, pred_species, true, pred

In [ ]:
def optimize_lmbda(df):
    
    # determine species names 
    species = df.columns.values[2:]

    # separate mono culture data 
    mono_dfs = []
    dfs = []
    treatments = []
    for treatment, df_i in df.groupby("Treatments"):
        # hyphen is only in community conditions
        if "-" in treatment:
            dfs.append(df_i)
            # save treatment name without the replicate identifier 
            treatments.append([treatment.split("_")[0]]*df_i.shape[0])
        else:
            mono_dfs.append(df_i)
    treatments = np.concatenate(treatments)
    unique_treatments = np.unique(treatments)
    mono_df = pd.concat(mono_dfs)
    df = pd.concat(dfs)

    # init vector of prediction performances
    performances = np.zeros(len(lmbdas))
        
    # scan range of lmbdas
    for lmbda_idx, lmbda in enumerate(lmbdas):
    
        # init kfold object
        kf = KFold(n_splits=n_splits_2, shuffle=True, random_state=21)

        # keep track of all predictions
        all_exp_names = []
        all_pred_species = []
        all_true = []
        all_pred = []

        # run Kfold 
        for kf_idx, (train_index, test_index) in enumerate(kf.split(unique_treatments)):

            # get train df
            train_inds = np.in1d(treatments, unique_treatments[train_index])
            train_df = df.iloc[train_inds].copy()
            train_df = pd.concat((mono_df, train_df))

            # average replicates in the test_df
            test_df = []
            for test_treatment in unique_treatments[test_index]:
                # pull dataframe with all replicates of same test treatment 
                treatment_inds = np.in1d(treatments, test_treatment)
                df_treatment = df.iloc[treatment_inds].copy()

                # get set of unique measurement times
                treatment_times = np.unique(df_treatment.Time.values)

                # init dataframe to store averaged values
                avg_df = pd.DataFrame()
                avg_df['Treatments'] = [test_treatment]*len(treatment_times)
                avg_df['Time'] = treatment_times

                avg_data = np.zeros([len(treatment_times), len(species)])
                for i, time in enumerate(treatment_times):
                    avg_data[i] = df_treatment.iloc[df_treatment.Time.values==time][species].mean()
                avg_df[species] = avg_data
                test_df.append(avg_df)

            # combine averaged dataframes for test dataframe
            test_df = pd.concat(test_df)

            # init model 
            model = gLV(dataframe=train_df, 
                        species=species,
                        lmbda=lmbda)

            # fit to data 
            print(f"Inner Fold {kf_idx+1}, L1: {lmbda}")
            f = model.fit_rmse(epochs=n_epochs)

            # plot fitness to data
            exp_names, pred_species, true, pred = predict_df(model, test_df, species)
            
            # append predictions 
            all_exp_names = np.append(all_exp_names, exp_names)
            all_pred_species = np.append(all_pred_species, pred_species)
            all_true = np.append(all_true, true)
            all_pred = np.append(all_pred, pred)

        # show prediction performance of individual species
        r_values = []
        for sp in species:
            sp_inds = all_pred_species == sp
            r_values.append(linregress(all_true[sp_inds], all_pred[sp_inds]).rvalue)

        # save performance
        performances[lmbda_idx] = np.mean(r_values)
        
        # print performance and lmbda
        print(f"L1: {lmbda}, avg r: {np.mean(r_values)}")
        
    # return best lmbda
    return lmbdas[np.argmax(performances)]

In [ ]:
# run kfold for each file 
for file in files:
    strain = file.split("_")[0]
    
    # import data
    df = pd.read_csv(f"data/SET3_Thirdtrial/{file}")
    df.sort_values(by=["Treatments", "Time"], inplace=True)
    
    # make sure that conditions have at least one measurement
    dfs = []
    for treatment, df_t in df.groupby("Treatments"):
        if df_t.shape[0] > 1:
            dfs.append(df_t)
    df = pd.concat(dfs)

    # determine species names 
    species = df.columns.values[2:]

    # separate mono culture data 
    mono_dfs = []
    dfs = []
    treatments = []
    for treatment, df_i in df.groupby("Treatments"):
        # hyphen is only in community conditions
        if "-" in treatment:
            dfs.append(df_i)
            # save treatment name without the replicate identifier 
            treatments.append([treatment.split("_")[0]]*df_i.shape[0])
        else:
            mono_dfs.append(df_i)
    treatments = np.concatenate(treatments)
    unique_treatments = np.unique(treatments)
    mono_df = pd.concat(mono_dfs)
    df = pd.concat(dfs)

    # init kfold object
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=21)

    # keep track of all predictions
    all_exp_names = []
    all_pred_species = []
    all_true = []
    all_pred = []

    # run Kfold 
    for kf_idx, (train_index, test_index) in enumerate(kf.split(unique_treatments)):
        
        # get train df
        train_inds = np.in1d(treatments, unique_treatments[train_index])
        train_df = df.iloc[train_inds].copy()
        train_df = pd.concat((mono_df, train_df))
        
        # average replicates in the test_df
        test_df = []
        for test_treatment in unique_treatments[test_index]:
            # pull dataframe with all replicates of same test treatment 
            treatment_inds = np.in1d(treatments, test_treatment)
            df_treatment = df.iloc[treatment_inds].copy()
            
            # get set of unique measurement times
            treatment_times = np.unique(df_treatment.Time.values)
            
            # init dataframe to store averaged values
            avg_df = pd.DataFrame()
            avg_df['Treatments'] = [test_treatment]*len(treatment_times)
            avg_df['Time'] = treatment_times

            avg_data = np.zeros([len(treatment_times), len(species)])
            for i, time in enumerate(treatment_times):
                avg_data[i] = df_treatment.iloc[df_treatment.Time.values==time][species].mean()
            avg_df[species] = avg_data
            test_df.append(avg_df)
        
        # combine averaged dataframes for test dataframe
        test_df = pd.concat(test_df)
        
        # optimize lambda using kfold on training data (nested kfold)
        lmbda = optimize_lmbda(train_df)
        print(f"Strain {strain}, Outer Fold {kf_idx+1}, L1: {lmbda}")
        
        # init model 
        model = gLV(dataframe=train_df, 
                    species=species,
                    lmbda=lmbda)

        # fit to data 
        f = model.fit_rmse(epochs=n_epochs)

        # plot fitness to data
        exp_names, pred_species, true, pred = predict_df(model, test_df, species)

        # append predictions 
        all_exp_names = np.append(all_exp_names, exp_names)
        all_pred_species = np.append(all_pred_species, pred_species)
        all_true = np.append(all_true, true)
        all_pred = np.append(all_pred, pred)

        # save prediction results to a .csv
        kfold_df = pd.DataFrame()
        kfold_df['Treatments'] = all_exp_names
        kfold_df['species'] = all_pred_species
        kfold_df['true'] = all_true
        kfold_df['pred'] = all_pred
        kfold_df.to_csv(f"kfold/{strain}_{n_splits}_fold_3.csv", index=False)
        
    # show prediction performance of individual species
    for sp in species:
        sp_inds = all_pred_species == sp
        R = linregress(all_true[sp_inds], all_pred[sp_inds]).rvalue
        plt.scatter(all_true[sp_inds], all_pred[sp_inds], label=f"{sp} " + "R={:.3f}".format(R))

    plt.xlabel("Measured OD")
    plt.ylabel("Predicted OD")
    plt.legend()
    plt.title(strain)
    plt.savefig(f"figures/{strain}_{n_splits}_fold_3.pdf", dpi=300)
    plt.show()